In [228]:
## Global Options
rate_limit_seconds = 3  # API Rate limit in seconds
rate_limit_timeout = 0  # When the rate limit will expire

def divider(): 
    print("-" * 50)


In [ ]:
import os
import time
from openai import OpenAI

if not os.environ.get("OPENAI_API"):
    raise ValueError("OPENAI_API environment variable is not set. Please set it to your API key.")
else : 
    print("API Key is set.")

# -------------------------------------------------
# Function to generate a response 

## Models to test with gpt-5, gpt-5-chat-latest (chatty), gpt-5-nano (cheap reasoning, no temperature)
# seed=seed  # Not supported with client.responses.create

def generate_response(prompt, model="gpt-5-nano", temperature=0.7, max_tokens=200, top_p=1.0, debug=False):

    # Adjust parameters for models
    if model == "gpt-5-nano":
        if max_tokens < 1000 : 
            max_tokens = 1000    # gpt-5-nano requires at least 1000 tokens
        if temperature != None: 
            temperature = None  # gpt-5-nano does not support temperature
            if debug: print("Temperature Not Supported")
        if top_p != None: 
            top_p = None              # gpt-5-nano does not support top_p
            if debug: print("Top_p Not Supported")

    if debug:
        print(f"Generating response with model: {model}, temperature: {temperature}, max_tokens: {max_tokens}, top_p: {top_p}, ")

    # Rate limit
    global rate_limit_seconds
    global rate_limit_timeout
    now = time.time()
    dif = rate_limit_timeout - now   

    # Spinner 
    if (dif > 0):
        # Spinner Countdown   
        while dif > 0:
            print(f"\rRate limit {dif:5.3f}s", end='', flush=True)
            time.sleep(0.25)  # Update spinner every 0.25 seconds
            dif = rate_limit_timeout - time.time()
        print("\r                         \r", end='', flush=True)
        # /Spinner 

    # Set the Rate Limit    
    rate_limit_timeout = time.time() + rate_limit_seconds
    
    client = OpenAI(api_key=os.environ["OPENAI_API"],
                    )

    try:
        response = client.responses.create(
        model=model,
        input=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_output_tokens=max_tokens,
        top_p=top_p,  
        service_tier="flex",  
        safety_identifier="test_user",
        prompt_cache_key="test_user",
        )
    except Exception as e:
        print(f"Error generating response: {e}")
        return None
    if debug:
        print(f"Usage: {response.usage.input_tokens} + {response.usage.output_tokens} = {response.usage.total_tokens}\n")
    return response.output_text 

# /------------------------------------------------

# Test
if (True):  # Set to True to run examples
    print()
    prompt = "Testing 1, 2, 3, ..."
    response = generate_response(prompt, debug=True)
    print(f"Prompt: {prompt}\nResponse: {response}")
    

API Key is set.

Generating response with model: gpt-5-nano, temperature: None, max_tokens: 1000, top_p: None, 
Usage: 17 + 382 = 399

Prompt: Testing 1, 2, 3, ...
Response: Loud and clear. What would you like to test or do today? I can help with Q&A, writing, summarizing, coding, translating, or analyzing data. If you want to test image input, you can share an image and I’ll describe or analyze it.


In [230]:
if (True):  # Set to True to run examples
    # Example: Basic prompt
    basic_prompt = "Explain AI in one sentence."
    response = generate_response(basic_prompt)
    print("Basic Prompt")
    print(f"Prompt: {basic_prompt}\nResponse: {response}\n")
    divider()

    # Example: Zero-shot, few-shot, and CoT prompts
    zero_shot_prompt = "Classify this sentiment: 'I love this product!'"
    zero_response = generate_response(zero_shot_prompt, temperature=0.5)
    print("Zero-shot Prompt")
    print(f"Prompt: {zero_shot_prompt}\nResponse: {zero_response}\n")
    divider()

    # Few-shot example
    few_shot_prompt = """
    Example 1: Text: 'Great movie.' Sentiment: Positive.
    Example 2: Text: 'Terrible service.' Sentiment: Negative.
    Classify: 'Okay experience.'
    """
    few_response = generate_response(few_shot_prompt, top_p=0.9)  # Now supported
    print("Few-shot Prompt")
    print(f"Prompt: {few_shot_prompt}\nResponse: {few_response}\n")
    divider()

    # Chain of Thought (CoT) example
    cot_prompt = "Solve: What is 15% of 200? Think step by step."
    cot_response = generate_response(cot_prompt, temperature=0.2)
    print("CoT Prompt")
    print(f"Prompt: {cot_prompt}\nResponse: {cot_response}\n")
    divider()


Basic Prompt
Prompt: Explain AI in one sentence.
Response: Artificial intelligence is the field of computer science that builds systems capable of learning from data, reasoning about problems, and performing tasks that normally require human intelligence.

--------------------------------------------------
Zero-shot Prompt         
Prompt: Classify this sentiment: 'I love this product!'
Response: Positive

--------------------------------------------------
Few-shot Prompt          
Prompt: 
    Example 1: Text: 'Great movie.' Sentiment: Positive.
    Example 2: Text: 'Terrible service.' Sentiment: Negative.
    Classify: 'Okay experience.'
    
Response: Neutral. The phrase expresses a middling sentiment, not clearly positive or negative.

--------------------------------------------------
CoT Prompt               
Prompt: Solve: What is 15% of 200? Think step by step.
Response: Step-by-step:

- Convert 15% to a decimal: 15% = 0.15
- Multiply by 200: 0.15 × 200 = 30

Alternative quick 

In [231]:
print("End of recent edits")

End of recent edits
